# Titanic Baseline Model (exp001)

EDAの結果を基にLightGBMでベースラインモデルを構築

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import japanize_matplotlib

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.rcParams['font.family'] = 'IPAexGothic'
sns.set_palette('husl')

## データの読み込み

In [ ]:
# データの読み込み
train_df = pd.read_csv('../../data/train.csv')
test_df = pd.read_csv('../../data/test.csv')

print(f"訓練データ: {train_df.shape}")
print(f"テストデータ: {test_df.shape}")

# 全データを結合（特徴量エンジニアリングのため）
all_data = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
print(f"全データ: {all_data.shape}")

## 特徴量エンジニアリング

EDAで効果的だった特徴量を作成

In [ ]:
def feature_engineering(df):
    """
    特徴量エンジニアリングを実行
    """
    df = df.copy()
    
    # 1. 称号（Title）の抽出
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # 称号のグルーピング
    title_mapping = {
        'Mr': 'Mr',
        'Miss': 'Miss',
        'Mrs': 'Mrs',
        'Master': 'Master',
        'Dr': 'Rare',
        'Rev': 'Rare',
        'Col': 'Rare',
        'Major': 'Rare',
        'Mlle': 'Miss',
        'Countess': 'Rare',
        'Ms': 'Miss',
        'Lady': 'Rare',
        'Jonkheer': 'Rare',
        'Don': 'Rare',
        'Dona': 'Rare',
        'Mme': 'Mrs',
        'Capt': 'Rare',
        'Sir': 'Rare'
    }
    df['Title'] = df['Title'].map(title_mapping)
    df['Title'] = df['Title'].fillna('Other')
    
    # 2. 家族サイズ
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # 3. 一人旅フラグ
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # 4. 家族サイズのカテゴリ化
    df['FamilySizeGroup'] = 'Medium'
    df.loc[df['FamilySize'] == 1, 'FamilySizeGroup'] = 'Alone'
    df.loc[df['FamilySize'] >= 5, 'FamilySizeGroup'] = 'Large'
    
    # 5. 年齢のビニング
    # 年齢の欠損値を称号で補完
    age_by_title = df.groupby('Title')['Age'].median()
    for title in age_by_title.index:
        df.loc[(df['Age'].isnull()) & (df['Title'] == title), 'Age'] = age_by_title[title]
    
    # 年齢をカテゴリ化
    df['AgeBin'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 120], labels=['Child', 'Teenager', 'Adult', 'Elder'])
    
    # 6. 運賃のビニング
    # 運賃の欠損値をクラスの中央値で補完
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df['FareBin'] = pd.qcut(df['Fare'], q=4, labels=['Low', 'Medium', 'High', 'VeryHigh'])
    
    # 7. Cabinの有無
    df['HasCabin'] = df['Cabin'].notna().astype(int)
    
    # 8. 乗船港の欠損値補完
    df['Embarked'] = df['Embarked'].fillna('S')  # 最頻値で補完
    
    # 9. 性別×クラスの組み合わせ特徴量
    df['Sex_Pclass'] = df['Sex'] + '_' + df['Pclass'].astype(str)
    
    # 10. チケット番号の特徴量
    df['TicketPrefix'] = df['Ticket'].str.extract('([A-Za-z]+)', expand=False)
    df['TicketPrefix'] = df['TicketPrefix'].fillna('None')
    df['TicketNumber'] = df['Ticket'].str.extract('([0-9]+)', expand=False)
    df['TicketNumber'] = df['TicketNumber'].fillna(0).astype(int)
    df['HasTicketPrefix'] = (df['TicketPrefix'] != 'None').astype(int)
    
    return df

# 特徴量エンジニアリングを実行
all_data = feature_engineering(all_data)
print("特徴量エンジニアリング完了")
print(f"新しい特徴量数: {all_data.shape[1]}")

In [ ]:
# 新しい特徴量の確認
print("新しく作成した特徴量:")
new_features = ['Title', 'FamilySize', 'IsAlone', 'FamilySizeGroup', 'AgeBin', 
                'FareBin', 'HasCabin', 'Sex_Pclass', 'TicketPrefix', 'HasTicketPrefix']

for feature in new_features:
    if feature in all_data.columns:
        print(f"\n{feature}:")
        print(all_data[feature].value_counts())

## データの前処理

In [ ]:
# カテゴリカル変数をエンコード
categorical_features = ['Sex', 'Embarked', 'Title', 'FamilySizeGroup', 'AgeBin', 'FareBin', 'Sex_Pclass', 'TicketPrefix']

# Label Encoding
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    all_data[feature] = le.fit_transform(all_data[feature].astype(str))
    label_encoders[feature] = le

print("ラベルエンコーディング完了")

In [ ]:
# 使用する特徴量を選択
features_to_use = [
    'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked',
    'Title', 'FamilySize', 'IsAlone', 'FamilySizeGroup', 'AgeBin', 'FareBin',
    'HasCabin', 'Sex_Pclass', 'HasTicketPrefix'
]

# 訓練・テストデータに分割
train_data = all_data[:len(train_df)].copy()
test_data = all_data[len(train_df):].copy()

X = train_data[features_to_use]
y = train_data['Survived']
X_test = test_data[features_to_use]

print(f"訓練データ特徴量: {X.shape}")
print(f"テストデータ特徴量: {X_test.shape}")
print(f"使用特徴量: {features_to_use}")

## モデルの訓練と評価

In [ ]:
# 訓練・検証データの分割
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"訓練データ: {X_train.shape}")
print(f"検証データ: {X_val.shape}")

In [ ]:
# LightGBMモデルの設定
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

# LightGBMデータセットの作成
train_dataset = lgb.Dataset(X_train, label=y_train)
val_dataset = lgb.Dataset(X_val, label=y_val, reference=train_dataset)

# モデルの訓練
model = lgb.train(
    lgb_params,
    train_dataset,
    num_boost_round=1000,
    valid_sets=[train_dataset, val_dataset],
    valid_names=['train', 'valid'],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(100)]
)

print("\nモデル訓練完了")

In [ ]:
# 検証データでの予測
y_pred = model.predict(X_val, num_iteration=model.best_iteration)
y_pred_binary = (y_pred > 0.5).astype(int)

# 精度評価
accuracy = accuracy_score(y_val, y_pred_binary)
print(f"検証精度: {accuracy:.4f}")

print("\n分類レポート:")
print(classification_report(y_val, y_pred_binary))

In [ ]:
# 混同行列の可視化
plt.figure(figsize=(6, 4))
cm = confusion_matrix(y_val, y_pred_binary)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('混同行列')
plt.ylabel('実際のラベル')
plt.xlabel('予測ラベル')
plt.show()

## 特徴量重要度

In [ ]:
# 特徴量重要度の取得
importance = model.feature_importance(importance_type='gain')
feature_importance = pd.DataFrame({
    'feature': features_to_use,
    'importance': importance
}).sort_values('importance', ascending=False)

# 特徴量重要度の可視化
plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
plt.title('特徴量重要度（Top 15）')
plt.xlabel('重要度')
plt.tight_layout()
plt.show()

print("特徴量重要度:")
print(feature_importance)

## クロスバリデーション

In [ ]:
# 5-fold クロスバリデーション
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    # データセット作成
    train_dataset = lgb.Dataset(X_train_fold, label=y_train_fold)
    val_dataset = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_dataset)
    
    # モデル訓練
    fold_model = lgb.train(
        lgb_params,
        train_dataset,
        num_boost_round=1000,
        valid_sets=[val_dataset],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )
    
    # 予測と評価
    y_pred_fold = fold_model.predict(X_val_fold, num_iteration=fold_model.best_iteration)
    y_pred_fold_binary = (y_pred_fold > 0.5).astype(int)
    fold_score = accuracy_score(y_val_fold, y_pred_fold_binary)
    cv_scores.append(fold_score)
    
    print(f"Fold {fold+1}: {fold_score:.4f}")

print(f"\nCV平均スコア: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores)*2:.4f})")

## テストデータでの予測

In [ ]:
# 全訓練データでモデルを再訓練
full_train_dataset = lgb.Dataset(X, label=y)
final_model = lgb.train(
    lgb_params,
    full_train_dataset,
    num_boost_round=model.best_iteration
)

# テストデータで予測
test_predictions = final_model.predict(X_test)
test_predictions_binary = (test_predictions > 0.5).astype(int)

print(f"テスト予測完了")
print(f"生存予測数: {test_predictions_binary.sum()}")
print(f"死亡予測数: {len(test_predictions_binary) - test_predictions_binary.sum()}")
print(f"生存率: {test_predictions_binary.mean():.4f}")

## 提出ファイルの作成

In [ ]:
# 提出ファイルの作成
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': test_predictions_binary
})

# results/exp001ディレクトリを作成
import os
os.makedirs('../../results/exp001', exist_ok=True)

# ファイルを保存
submission.to_csv('../../results/exp001/submission.csv', index=False)
print("提出ファイルを保存しました: ../../results/exp001/submission.csv")

# 提出ファイルの確認
print("\n提出ファイルの最初の10行:")
print(submission.head(10))

## 実験結果のまとめ

### 使用した特徴量
1. **基本特徴量**: Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
2. **エンジニアリング特徴量**: 
   - Title（称号）
   - FamilySize（家族サイズ）
   - IsAlone（一人旅フラグ）
   - FamilySizeGroup（家族サイズカテゴリ）
   - AgeBin（年齢ビン）
   - FareBin（運賃ビン）
   - HasCabin（客室有無）
   - Sex_Pclass（性別×クラス）
   - HasTicketPrefix（チケット番号プレフィックス有無）

### モデル
- **アルゴリズム**: LightGBM
- **ハイパーパラメータ**: learning_rate=0.05, num_leaves=31など

### 結果
- **検証精度**: 表示された精度を参照
- **CV平均スコア**: 表示されたスコアを参照

### 次のステップ
1. ハイパーパラメータチューニング
2. アンサンブル学習
3. より複雑な特徴量の作成
4. 外部データの活用